1. dividend growth twice the rate of inflation over last 3 years
2. flat or declinging payout ratio over last 3 years
    - Payout Ratio = (Dividends per Share (DPS) / Earnings per Share (EPS)) x 100
    - Payout Ratio = (Total Dividends Paid / Net Income) x 100
3. 3 year average payout ratio less than 60%
4. debt to equity ratio under 1.5
    - Debt/Equity Ratio = Total Liabilities / Shareholders' Equity
5. maintain net margins over 7% 
    - Net Margin = Net Income / Total Revenues
6. ROIC level exceeds 20%
    - Net Operating Profit After Tax (NOPAT)/Invested Capital = ROIC
7. decreasing diluted share count

In [1]:
# Note to import from .py files, must follow structure
# from <.py filename excluding '.py'> import <class name>
# Optionslam creds: aspringfastlaner Options2018

# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.dates as dates
# import matplotlib.ticker as ticker
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import sched, time
import pandas_datareader as datareader
from pandas_datareader.data import Options
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt

from pandas.io.json import json_normalize
import urllib.request as req
import pandas_market_calendars as mcal
nyse = mcal.get_calendar('NYSE')
from helpers import *

import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *
import plotly
import os


from scipy.stats import norm as norm
from yahoo_query import *

# Alpha Vantage API Key
# 5HZEUI5AFJB06BUK

# ts = TimeSeries(key='5HZEUI5AFJB06BUK', output_format='pandas')
# data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')
# data['close'].plot()
# plt.title('Intraday Times Series for the MSFT stock (1 min)')
# For intraday
# https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=MSFT&interval=1min&apikey=d5HZEUI5AFJB06BUK&datatype=csv

# For daily
# https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=5HZEUI5AFJB06BUK&datatype=csv

In [3]:
# '''
# functions list:

#     maturities(dt.datetime()) --> [float(front_wgt), float(back_wgt)]

#     optionslam_scrape(str[ticker]) --> dict[earnings]

#     yahoo_table_parse(str[raw_html_table]) --> DataFrame[ticker]

#     yahoo_earnings(dt.datetime()) --> DataFrame[earnings_on_date]

#     fundamentals(str[ticker]) --> DataFrame[stock_fundamentals]

#     get_fundas(list[ticker_lst]) --> DataFrame[stock_fundamentals]

#     historical_data(str[ticker], int[day_number], int[rolling_window], outsize[str]) --> DataFrame[daily_stock_data]

#     current_volatility(list[ticker_lst], int[roll]) --> DataFrame[stock_volatilities]

#     all_options(str[ticker], bool[greeks]) --> DataFrame[options_chains]

#     earnings_condor(str[ticker], int[max_gap], int[dte_thresh], float[|money_thresh| <= 1]) -- DataFrame[condors], DataFrame[puts], DataFrame[calls]

#     write_excel(str[filename], list[str[sheetnames]], list[dataframes]) --> void()

#     curr_stock_data(str[ticker]) --> DataFrame[stock_info]

#     curr_batch_quotes(list_of_string[tickers]) --> DataFrame[stock_info]

#     past_earnings(str[ticker]) --> DataFrame[earnings_info]

#     earnings_history(str[ticker]) --> [DataFrame[earnings_estimate], DataFrame[past_earnings], DataFrame[earnings_estimate_change]]

#     av_data(str[ticker]) --> DataFrame[ticker_open, ticker_close]

#     av_batch(list_of_str[tickers]) --> DataFrame[tickers_closes]

#     check_mkt_corr(int[corr_rolling_window],int[plot_window]) --> DataFrame[rolling_corr]

#     vvix_check() --> DataFrame[VVIX Data]

#     earnings_longs(list_of_str[ticker], float[bid_ask_spread]) --> DataFrame[option_chains]

#     all_options_v2(str[ticker], int[dte_ub], int[dte_lb], float[moneyness]) --> DataFrame[option_chains]

#     yahoo_options_query(str[ticker], int[dte_ub], int[dte_lb]) --> DataFrame[option_chains]

#     greek_calc(DataFrame[option_chain], str[prem_price_use], str[day_format], float[interest_rate], float[dividend_rate])

#     price_sim(DataFrame[options_df], float[price_change], float[vol_change], int[days_change], str[output = 'All'],
#               str[skew = 'flat'], str[day_format = 'trading'], float[interest_rate = 0.0193], float[dividend_rate = 0],
#               float[prem_price_use = 'Mid'])


#     position_sim(DataFrame[position_df], list_of_int[holdings], int[shares],
#                  float[price_change], float[vol_change], int[dte_change], str[output = 'All'],
#                  str[skew = 'flat'], str[prem_price_use = 'Mid'], str[day_format = 'trading'], 
#                  float[interest_rate = 0.0193], float[dividend_rate = 0])

#     yahoo_fundamentals(list_of_str[tickers]) --> DataFrame[fundamentals]

# '''

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Single Name Testing\\Single-Name-Testing\\Names')

tsx = pd.read_table('TSX.txt')
tsxv = pd.read_table('TSXV.txt')
amex = pd.read_table('AMEX.txt')
nasdaq = pd.read_table('NASDAQ.txt')
nyse = pd.read_table('NYSE.txt')

us_names = pd.concat([amex,nasdaq, nyse], axis = 0)[['Symbol']].drop_duplicates()
cad_names = pd.concat([tsx,tsxv], axis = 0)[['Symbol']].drop_duplicates()

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Single Name Testing\\Single-Name-Testing\\DGI Pulling')

stock_list = pd.read_csv('companylist.csv')

start_time = time.time()

def pull_data(ticker):
    yahoo_data = yahoo_query(ticker,dt.datetime(2018,1,1))
    yahoo_data.full_info_query()
    
    def create_dgi(balanceSheet, incomeStatement, cashFlowStatement):
        dgi_data = balanceSheet[['totalLiab',
                                 'totalStockholderEquity',
                                 'longTermInvestments',
                                 'shortTermInvestments']]
        
        dgi_data = dgi_data.join(incomeStatement[['netIncome',
                                                  'operatingIncome',
                                                  'totalRevenue']])
        
        dgi_data = dgi_data.join(cashFlowStatement[['dividendsPaid',
                                                    'investments']])
        dgi_data['dividendsPaid'] = -dgi_data['dividendsPaid']

        dgi_data['payoutRatio'] = dgi_data.dividendsPaid/dgi_data.netIncome
        dgi_data['debtEquityRatio'] = dgi_data.totalLiab/dgi_data.totalStockholderEquity
        dgi_data['netMargin'] = dgi_data.netIncome/dgi_data.totalRevenue
        dgi_data['roic'] = dgi_data.operatingIncome/(dgi_data.longTermInvestments + dgi_data.shortTermInvestments)
        
        dgi_data = dgi_data[['payoutRatio','debtEquityRatio','netMargin','roic','dividendsPaid']]
        dgi_data['Underyling'] = ticker
        return dgi_data
    
    dgi_annual = create_dgi(yahoo_data.balanceSheetAnnual, yahoo_data.incomeStatementAnnual, 
                            yahoo_data.cashFlowStatementAnnual)

    ### separate df for current key measures
    keyMetrics = yahoo_data.profile.join(yahoo_data.keyStats).join(yahoo_data.finData, rsuffix = '_finData')
    
    return (dgi_annual, keyMetrics)

def download_yahoo_data(ticker_list, retries = 10, lookback_years = 3):

    earnings_lst = []
    keyStats_lst = []
    dgi_lst = []

    item_counter = 0
    total_length = len(ticker_list)
    failed_list = []

    for ticker in ticker_list:
        try:
            curr_earnings, curr_keyStats = pull_data(ticker)
            earnings_lst.append(curr_earnings)
            keyStats_lst.append(curr_keyStats)
            
            dgi_df = curr_earnings[['Underyling']]#pd.DataFrame(index = [ticker])
            dgi_df['quarter'] = pd.to_datetime(curr_earnings.index)
            dgi_df['payoutChange'] = curr_earnings.payoutRatio.sort_index().pct_change().tail(lookback_years).mean()
            dgi_df['divGrowth'] = curr_earnings.dividendsPaid.sort_index().pct_change().tail(lookback_years).mean()
            dgi_df['payoutAvg'] = curr_earnings.payoutRatio.sort_index().tail(lookback_years).mean()
            dgi_df['debtEquityAvg'] = curr_earnings.debtEquityRatio.sort_index().tail(lookback_years).mean()
            dgi_df['netMarginAvg'] = curr_earnings.netMargin.sort_index().tail(lookback_years).mean()
            #dgi_df['roicAvg'] = curr_earnings.roic.sort_index().tail(lookback_years).mean()

            dgi_lst.append(dgi_df)
            print('Accepted: {}'.format(ticker))
        except:
            for i in range(retries):
                try:
                    curr_earnings, curr_keyStats = pull_data(ticker)
                    earnings_lst.append(curr_earnings)
                    keyStats_lst.append(curr_keyStats)
                    
                    dgi_df = curr_earnings[['Underyling']] # pd.DataFrame(index = [ticker])
                    dgi_df['quarter'] = pd.to_datetime(curr_earnings.index)
                    dgi_df['payoutChange'] = curr_earnings.payoutRatio.sort_index().pct_change().tail(lookback_years).mean()
                    dgi_df['divGrowth'] = curr_earnings.dividendsPaid.sort_index().pct_change().tail(lookback_years).mean()
                    dgi_df['payoutAvg'] = curr_earnings.payoutRatio.sort_index().tail(lookback_years).mean()
                    dgi_df['debtEquityAvg'] = curr_earnings.debtEquityRatio.sort_index().tail(lookback_years).mean()
                    dgi_df['netMarginAvg'] = curr_earnings.netMargin.sort_index().tail(lookback_years).mean()
                    #dgi_df['roicAvg'] = curr_earnings.roic.sort_index().tail(lookback_years).mean()

                    dgi_lst.append(dgi_df)
                    print('Accepted: {}'.format(ticker))
                except:
                    continue
            print('Failed: {}'.format(ticker))
            failed_list.append(ticker)

        item_counter += 1
        print('{0:.2f}% Completed'.format(item_counter/total_length*100))
        print('{} total failures'.format(len(failed_list)))
    try:
        earnings_df = pd.concat(earnings_lst, axis = 0)
        # earnings_df = earnings_df.reset_index()[earnings_df.columns]
        keyStats_df = pd.concat(keyStats_lst, axis = 0)
        dgi_df = pd.concat(dgi_lst, axis = 0)

        return earnings_df, keyStats_df, dgi_df, failed_list
    except:
        return failed_list


In [ ]:
earnings_df, keyStats_df, dgi_df, failed_list = download_yahoo_data(cad_names['Symbol'].tolist(),10, 3)

earnings_df.fillna(0).to_csv('cad_dgi_raw.csv')
keyStats_df.to_csv('cad_keystats.csv')
dgi_df.fillna(0).to_csv('cad_dgi_scores.csv')

Failed: AAB.TO
0.03% Completed
1 total failures
Failed: AAV.TO
0.05% Completed
2 total failures


C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:134: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:135: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#i

Accepted: ABT.TO
0.08% Completed
2 total failures
Failed: ABX.TO
0.10% Completed
3 total failures
Failed: AC.TO
0.13% Completed
4 total failures
Failed: ACB.TO
0.15% Completed
5 total failures
Failed: ACB.WT.TO
0.18% Completed
6 total failures
Failed: ACD.TO
0.20% Completed
7 total failures
Failed: ACO.X.TO
0.23% Completed
8 total failures
Failed: ACO.Y.TO
0.26% Completed
9 total failures
Failed: ACQ.TO
0.28% Completed
10 total failures
Failed: ACR.UN.TO
0.31% Completed
11 total failures
Failed: ACZ.UN.TO
0.33% Completed
12 total failures
Failed: AD.TO
0.36% Completed
13 total failures
Failed: ADC.UN.TO
0.38% Completed
14 total failures
Failed: ADN.TO
0.41% Completed
15 total failures
Failed: ADW.A.TO
0.43% Completed
16 total failures
Failed: ADW.B.TO
0.46% Completed
17 total failures
Failed: AEF.TO
0.49% Completed
18 total failures
Failed: AEF.WT.TO
0.51% Completed
19 total failures
Failed: AEM.TO
0.54% Completed
20 total failures
Failed: AEZS.TO
0.56% Completed
21 total failures
Fail

Failed: BCE.PR.H.TO
4.14% Completed
159 total failures
Failed: BCE.PR.I.TO
4.17% Completed
160 total failures
Failed: BCE.PR.J.TO
4.20% Completed
161 total failures
Failed: BCE.PR.K.TO
4.22% Completed
162 total failures
Failed: BCE.PR.L.TO
4.25% Completed
163 total failures
Failed: BCE.PR.M.TO
4.27% Completed
164 total failures
Failed: BCE.PR.N.TO
4.30% Completed
165 total failures
Failed: BCE.PR.O.TO
4.32% Completed
166 total failures
Failed: BCE.PR.Q.TO
4.35% Completed
167 total failures
Failed: BCE.PR.R.TO
4.37% Completed
168 total failures
Failed: BCE.PR.S.TO
4.40% Completed
169 total failures
Failed: BCE.PR.T.TO
4.43% Completed
170 total failures
Failed: BCE.PR.Y.TO
4.45% Completed
171 total failures
Failed: BCE.PR.Z.TO
4.48% Completed
172 total failures
Failed: BCI.TO
4.50% Completed
173 total failures
Failed: BDI.TO
4.53% Completed
174 total failures
Accepted: BDT.TO
4.55% Completed
174 total failures
Failed: BEI.UN.TO
4.58% Completed
175 total failures
Failed: BEK.B.TO
4.60% Co

Failed: CBT.UN.TO
8.11% Completed
310 total failures
Failed: CCA.TO
8.14% Completed
311 total failures
Failed: CCI.UN.TO
8.16% Completed
312 total failures
Failed: CCL.A.TO
8.19% Completed
313 total failures
Failed: CCL.B.TO
8.21% Completed
314 total failures
Failed: CCM.TO
8.24% Completed
315 total failures
Failed: CCO.TO
8.26% Completed
316 total failures
Failed: CCS.PR.C.TO
8.29% Completed
317 total failures
Failed: CCX.TO
8.31% Completed
318 total failures
Failed: CCZ.TO
8.34% Completed
319 total failures
Failed: CCZ.PR.A.TO
8.37% Completed
320 total failures
Failed: CDAY.TO
8.39% Completed
321 total failures
Failed: CDD.UN.TO
8.42% Completed
322 total failures
Failed: CDH.TO
8.44% Completed
323 total failures
Failed: CDV.TO
8.47% Completed
324 total failures
Failed: CDZ.TO
8.49% Completed
325 total failures
Failed: CED.TO
8.52% Completed
326 total failures
Failed: CEE.TO
8.54% Completed
327 total failures
Failed: CEF.TO
8.57% Completed
328 total failures
Failed: CEF.U.TO
8.60% Com

In [ ]:
filterd_dgi = dgi_df[(dgi_df['debtEquityAvg'] <= 1.5) &
       (dgi_df['divGrowth'] >= 0.03) &
       (dgi_df['payoutChange'] <= 0) &
       (dgi_df['payoutAvg'] <= 0.6) & (dgi_df['payoutAvg'] >= 0) &
       (dgi_df['netMarginAvg'] >= 0.07) &
       (dgi_df['roicAvg'] >= 0.2)].sort_values('divGrowth', ascending = False)

filterd_dgi.to_csv('filtered_dgi_names.csv')

In [ ]:
# lookback_years = 3
# dgi_list = []

# for ticker in earnings_df['Underlying'].drop_duplicates().tolist():
#     curr_earnings = earnings_df[earnings_df['Underlying'] == ticker]

#     dgi_df = pd.DataFrame(index = [ticker])
#     dgi_df['payoutChange'] = curr_earnings.payoutRatio.sort_index().pct_change().tail(lookback_years).mean()
#     dgi_df['divGrowth'] = curr_earnings.dividendsPaid.sort_index().pct_change().tail(lookback_years).mean()
#     dgi_df['payoutAvg'] = curr_earnings.payoutRatio.sort_index().tail(lookback_years).mean()
#     dgi_df['debtEquityAvg'] = curr_earnings.debtEquityRatio.sort_index().tail(lookback_years).mean()
#     dgi_df['netMarginAvg'] = curr_earnings.netMargin.sort_index().tail(lookback_years).mean()
#     dgi_df['roicAvg'] = curr_earnings.roic.sort_index().tail(lookback_years).mean()
#     dgi_list.append(dgi_df)
